# **Homework 2-1 Phoneme Classification**

Author: Chih-Yuan Chuang (r09921006)

## The DARPA TIMIT Acoustic-Phonetic Continuous Speech Corpus (TIMIT)
The TIMIT corpus of reading speech has been designed to provide speech data for the acquisition of acoustic-phonetic knowledge and for the development and evaluation of automatic speech recognition systems.

This homework is a multiclass classification task, 
we are going to train a deep neural network classifier to predict the phonemes for each frame from the speech corpus TIMIT.

link: https://academictorrents.com/details/34e2b78745138186976cbc27939b1b34d18bd5b3

## Download Data
Download data from google drive, then unzip it.

You should have `timit_11/train_11.npy`, `timit_11/train_label_11.npy`, and `timit_11/test_11.npy` after running this block.<br><br>
`timit_11/`
- `train_11.npy`: training data<br>
- `train_label_11.npy`: training label<br>
- `test_11.npy`:  testing data<br><br>

**notes: if the google drive link is dead, you can download the data directly from Kaggle and upload it to the workspace**




In [ ]:
from google.colab import drive
drive.mount('/content/drive')
#/content/drive/MyDrive/windows備份/C_Users_Alvin_Documents/~碩班/Course/109-2/機器學習_李宏毅/Week 3/HW2

Mounted at /content/drive


In [ ]:
#!gdown --id '1HPkcmQmFGu-3OknddKIa5dNDsR05lIQR' --output data.zip
!cp '/content/drive/MyDrive/windows備份/C_Users_Alvin_Documents/~碩班/Course/109-2/機器學習_李宏毅/Week 3/HW2/timit_11_v2.zip' './data.zip'
!unzip data.zip
!ls 

Archive:  data.zip
   creating: timit_11/
  inflating: timit_11/train_11.npy   
  inflating: timit_11/test_11.npy    
  inflating: timit_11/train_label_11.npy  
data.zip  drive  sample_data  timit_11


## Preparing Data
Load the training and testing data from the `.npy` file (NumPy array).

In [ ]:
import numpy as np

print('Loading data ...')

data_root='./timit_11/'
train = np.load(data_root + 'train_11.npy')
train_label = np.load(data_root + 'train_label_11.npy')
test = np.load(data_root + 'test_11.npy')

print('Size of training data: {}'.format(train.shape))
print('Size of testing data: {}'.format(test.shape))

Loading data ...
Size of training data: (1229932, 429)
Size of testing data: (451552, 429)


In [ ]:
train_mean = np.mean(train, axis=0, keepdims=True)
train_std = np.std(train, axis=0, keepdims=True)
print('Size of training data mean: {}'.format(train_mean.shape))
print('Size of training data std: {}'.format(train_std.shape))

Size of training data mean: (1, 429)
Size of training data std: (1, 429)


In [ ]:
# Normalization
# If "train = (train - train_mean) / train_std", memory usage is too large somehow???
train = (train - train_mean)
train = train / train_std
test = (test - train_mean)
test = test / train_std
print('Size of training data after normalization: {}'.format(train.shape))
print('Size of testining data after normalization: {}'.format(test.shape))
import gc
del train_mean, train_std
gc.collect()

Size of training data after normalization: (1229932, 429)
Size of testining data after normalization: (451552, 429)


109

## Create Dataset

In [ ]:
import torch
from torch.utils.data import Dataset

class TIMITDataset(Dataset):
    def __init__(self, X, y=None):
        self.data = torch.from_numpy(X).float()
        if y is not None:
            y = y.astype(np.int)
            self.label = torch.LongTensor(y)
        else:
            self.label = None

    def __getitem__(self, idx):
        if self.label is not None:
            return self.data[idx], self.label[idx]
        else:
            return self.data[idx]

    def __len__(self):
        return len(self.data)


Split the labeled data into a training set and a validation set, you can modify the variable `VAL_RATIO` to change the ratio of validation data.

In [ ]:
VAL_RATIO = 0.1#0.2

percent = int(train.shape[0] * (1 - VAL_RATIO))
train_x, train_y, val_x, val_y = train[:percent], train_label[:percent], train[percent:], train_label[percent:]
print('Size of training set: {}'.format(train_x.shape))
print('Size of validation set: {}'.format(val_x.shape))

Size of training set: (1106938, 429)
Size of validation set: (122994, 429)


Create a data loader from the dataset, feel free to tweak the variable `BATCH_SIZE` here.

In [ ]:
BATCH_SIZE = 41#64

from torch.utils.data import DataLoader

train_set = TIMITDataset(train_x, train_y)
val_set = TIMITDataset(val_x, val_y)
train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True) #only shuffle the training data
val_loader = DataLoader(val_set, batch_size=BATCH_SIZE, shuffle=False)

Cleanup the unneeded variables to save memory.<br>

**notes: if you need to use these variables later, then you may remove this block or clean up unneeded variables later<br>the data size is quite huge, so be aware of memory usage in colab**

In [ ]:
#import gc

del train, train_label, train_x, train_y, val_x, val_y
gc.collect()

150

## Create Model

Define model architecture, you are encouraged to change and experiment with the model architecture.

In [ ]:
import torch
import torch.nn as nn

class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        self.layer1 = nn.Linear(429, 4096)#(429, 1024)
        self.layer1_1 = nn.Linear(4096, 2048)#(1024, 1024)
        self.layer1_2 = nn.Linear(2048, 1024)#(1024, 1024)
        self.layer1_2_2 = nn.Linear(1024, 1024)
        self.layer1_3 = nn.Linear(1024, 512)#(1024, 1024)
        # self.layer1_4 = nn.Linear(1024, 1024)
        # self.layer1_5 = nn.Linear(1024, 1024)
        self.layer2 = nn.Linear(512, 256)#(1024, 1024)#(1024, 512)
        self.layer3 = nn.Linear(256, 128)#(1024, 1024)#(512, 128)
        self.out = nn.Linear(128, 39)#(1024, 39)#(128, 39) 

        self.act_fn = nn.ReLU()#Sigmoid()#nn.ReLU()

        self.dropout = nn.Dropout(p=0.3)#p=0.2)

        self.bn1 = nn.BatchNorm1d(num_features=4096)
        self.bn1_1 = nn.BatchNorm1d(num_features=2048)
        self.bn1_2 = nn.BatchNorm1d(num_features=1024)
        self.bn1_2_2 = nn.BatchNorm1d(num_features=1024)
        self.bn1_3 = nn.BatchNorm1d(num_features=512)
        self.bn2 = nn.BatchNorm1d(num_features=256)
        self.bn3 = nn.BatchNorm1d(num_features=128)

    def forward(self, x):
        x = self.layer1(x)
        x = self.bn1(x)
        x = self.act_fn(x)
        x = self.dropout(x)

        x = self.layer1_1(x)
        x = self.bn1_1(x)
        x = self.act_fn(x)        
        x = self.dropout(x)

        x = self.layer1_2(x)
        x = self.bn1_2(x)
        x = self.act_fn(x)        
        x = self.dropout(x)

        x = self.layer1_2_2(x)
        x = self.bn1_2_2(x)
        x = self.act_fn(x)
        x = self.dropout(x)

        x = self.layer1_3(x)
        x = self.bn1_3(x)
        x = self.act_fn(x)        
        x = self.dropout(x)

        # x = self.layer1_4(x)
        # x = self.act_fn(x)        
        # x = self.dropout(x)

        # x = self.layer1_5(x)
        # x = self.act_fn(x)        
        # # x = self.dropout(x)

        x = self.layer2(x)
        x = self.bn2(x)
        x = self.act_fn(x)
        x = self.dropout(x)

        x = self.layer3(x)
        x = self.bn3(x)
        x = self.act_fn(x)
        x = self.dropout(x)

        x = self.out(x)
        
        return x

## Training

In [ ]:
#check device
def get_device():
  return 'cuda' if torch.cuda.is_available() else 'cpu'

Fix random seeds for reproducibility.

In [ ]:
# fix random seed
def same_seeds(seed):
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)  
    np.random.seed(seed)  
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

Feel free to change the training parameters here.

In [ ]:
# fix random seed for reproducibility
same_seeds(0)

# get device 
device = get_device()
print(f'DEVICE: {device}')

# training parameters
num_epoch = 40#20               # number of training epoch
learning_rate = 0.0001       # learning rate

# the path where checkpoint saved
model_path = './model.ckpt'

# create model, define a loss function, and optimizer
model = Classifier().to(device)
criterion = nn.CrossEntropyLoss() 
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
#optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
#optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=0.001)
#optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

DEVICE: cuda


In [ ]:
# start training

best_acc = 0.0
for epoch in range(num_epoch):
    train_acc = 0.0
    train_loss = 0.0
    val_acc = 0.0
    val_loss = 0.0

    # training
    model.train() # set the model to training mode
    for i, data in enumerate(train_loader):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad() 
        outputs = model(inputs) 
        batch_loss = criterion(outputs, labels)
        _, train_pred = torch.max(outputs, 1) # get the index of the class with the highest probability
        batch_loss.backward() 
        optimizer.step() 

        train_acc += (train_pred.cpu() == labels.cpu()).sum().item()
        train_loss += batch_loss.item()

    # validation
    if len(val_set) > 0:
        model.eval() # set the model to evaluation mode
        with torch.no_grad():
            for i, data in enumerate(val_loader):
                inputs, labels = data
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                batch_loss = criterion(outputs, labels) 
                _, val_pred = torch.max(outputs, 1) 
            
                val_acc += (val_pred.cpu() == labels.cpu()).sum().item() # get the index of the class with the highest probability
                val_loss += batch_loss.item()

            print('[{:03d}/{:03d}] Train Acc: {:3.6f} Loss: {:3.6f} | Val Acc: {:3.6f} loss: {:3.6f}'.format(
                epoch + 1, num_epoch, train_acc/len(train_set), train_loss/len(train_loader), val_acc/len(val_set), val_loss/len(val_loader)
            ))

            # if the model improves, save a checkpoint at this epoch
            if val_acc > best_acc:
                best_acc = val_acc
                torch.save(model.state_dict(), model_path)
                print('saving model with acc {:.3f}'.format(best_acc/len(val_set)))
    else:
        print('[{:03d}/{:03d}] Train Acc: {:3.6f} Loss: {:3.6f}'.format(
            epoch + 1, num_epoch, train_acc/len(train_set), train_loss/len(train_loader)
        ))

# if not validating, save the last epoch
if len(val_set) == 0:
    torch.save(model.state_dict(), model_path)
    print('saving model at last epoch')


[001/040] Train Acc: 0.554893 Loss: 1.526054 | Val Acc: 0.680830 loss: 1.032442
saving model with acc 0.681
[002/040] Train Acc: 0.632750 Loss: 1.220703 | Val Acc: 0.702514 loss: 0.935376
saving model with acc 0.703
[003/040] Train Acc: 0.658301 Loss: 1.126960 | Val Acc: 0.714962 loss: 0.895437
saving model with acc 0.715
[004/040] Train Acc: 0.673687 Loss: 1.069077 | Val Acc: 0.725198 loss: 0.855981
saving model with acc 0.725
[005/040] Train Acc: 0.686015 Loss: 1.023067 | Val Acc: 0.728694 loss: 0.837767
saving model with acc 0.729
[006/040] Train Acc: 0.695398 Loss: 0.987620 | Val Acc: 0.734011 loss: 0.816742
saving model with acc 0.734
[007/040] Train Acc: 0.704067 Loss: 0.956373 | Val Acc: 0.738922 loss: 0.807406
saving model with acc 0.739
[008/040] Train Acc: 0.711453 Loss: 0.928415 | Val Acc: 0.742776 loss: 0.793619
saving model with acc 0.743
[009/040] Train Acc: 0.717944 Loss: 0.905295 | Val Acc: 0.743134 loss: 0.782862
saving model with acc 0.743
[010/040] Train Acc: 0.72396

## Testing

Create a testing dataset, and load model from the saved checkpoint.

In [ ]:
# create testing dataset
test_set = TIMITDataset(test, None)
test_loader = DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=False)

# create model and load weights from checkpoint
model = Classifier().to(device)
model.load_state_dict(torch.load(model_path))

<All keys matched successfully>

Make prediction.

In [ ]:
predict = []
model.eval() # set the model to evaluation mode
with torch.no_grad():
    for i, data in enumerate(test_loader):
        inputs = data
        inputs = inputs.to(device)
        outputs = model(inputs)
        _, test_pred = torch.max(outputs, 1) # get the index of the class with the highest probability

        for y in test_pred.cpu().numpy():
            predict.append(y)

Write prediction to a CSV file.

After finish running this block, download the file `prediction.csv` from the files section on the left-hand side and submit it to Kaggle.

In [ ]:
with open('prediction.csv', 'w') as f:
    f.write('Id,Class\n')
    for i, y in enumerate(predict):
        if i < len(predict) - 1 and i > 0: # post processing
            if (predict[i-1] == predict[i+1] and y != predict[i-1]):
                y = predict[i-1]
            if (predict[i-1] != predict[i+1] and y != predict[i-1] and y != predict[i+1]):
                y = predict[i-1]
        f.write('{},{}\n'.format(i, y))

# **Reference**

Source: Ray Chen @ NTUEE (https://github.com/ga642381/ML2021-Spring/blob/main/HW02/HW02-1.ipynb)